## Import

In [27]:
import os
import cv2
from PIL import Image
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from tqdm import tqdm
import albumentations as A
from albumentations.pytorch import ToTensorV2

from torchvision import models
from torchsummary import summary


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

## Utils

In [20]:
# RLE 인코딩 함수
def rle_encode(mask):
    pixels = mask.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

## Custom Dataset

In [21]:
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None, infer=False):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.infer = infer

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path = self.data.iloc[idx, 1]
        image = cv2.imread(img_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.infer:
            if self.transform:
                image = self.transform(image=image)['image']
            return image
        
        mask_path = self.data.iloc[idx, 2]
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        mask[mask == 255] = 12 #배경을 픽셀값 12로 간주

        if self.transform:
            augmented = self.transform(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

## Data Loader

In [22]:
transform = A.Compose(
    [   
        A.Resize(224, 224),
        A.Normalize(),
        ToTensorV2()
    ]
)

dataset = CustomDataset(csv_file='./train_source.csv', transform=transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

valid_dataset = CustomDataset(csv_file='./val_source.csv', transform=transform)
valid_dataloader = DataLoader(valid_dataset, batch_size=16, shuffle=False, num_workers=4)


In [23]:
# 클래스(레이블) 수
num_classes = 12

# 클래스별 IoU를 계산하기 위한 함수
def calculate_iou_per_class(y_true, y_pred, class_id):
    intersection = np.sum((y_true == class_id) & (y_pred == class_id))
    union = np.sum((y_true == class_id) | (y_pred == class_id))
    iou = intersection / union if union > 0 else 0
    return iou

## Define Model

In [24]:
# U-Net의 기본 구성 요소인 Double Convolution Block을 정의합니다.
def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )

# 간단한 U-Net 모델 정의
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)

        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)

        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)

        self.conv_last = nn.Conv2d(64, 13, 1) # 12개 class + 1 background

    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)

        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)

        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)   

        x = self.dconv_down4(x)

        x = self.upsample(x)        
        x = torch.cat([x, conv3], dim=1)

        x = self.dconv_up3(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv2], dim=1)

        x = self.dconv_up2(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv1], dim=1)

        x = self.dconv_up1(x)

        out = self.conv_last(x)

        return out

In [61]:
# model 초기화
model = UNet().to(device)

# loss function과 optimizer 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


## Model Train

In [62]:
# training loop
for epoch in range(5):  # 5 에폭 동안 학습합니다.
          
    # 클래스별 IoU를 누적할 리스트 초기화
    class_ious = []
    
    # 학습
    model.train()
    epoch_loss = 0
    
    for images, masks in tqdm(dataloader):
        images = images.float().to(device)
        masks = masks.long().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks.squeeze(1))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        # validation 클래스별 IoU 계산
        outputs = torch.softmax(outputs, dim=1).cpu()
        outputs = torch.argmax(outputs, dim=1).numpy()

        for class_id in range(num_classes):
            iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
            class_ious.append(iou)

    # mIoU 계산
    train_mIoU = np.mean(class_ious)
    #mIoU = np.mean(class_ious)
    #print(f"Epoch {epoch+1}, Train Loss: {epoch_loss/len(dataloader)}, train mIoU Score: {mIoU:.4f}")
    
    # validation
    val_loss = 0
    class_ious = []  # 클래스별 IoU를 누적할 리스트 초기화
    with torch.no_grad():
        model.eval()
        for images, masks in tqdm(valid_dataloader):
            images = images.float().to(device)
            masks = masks.long().to(device)
            outputs = model(images)

            # validation loss 계산
            val_loss += criterion(outputs, masks.squeeze(1)).item()

            # validation 클래스별 IoU 계산
            outputs = torch.softmax(outputs, dim=1).cpu()
            outputs = torch.argmax(outputs, dim=1).numpy()

            for class_id in range(num_classes):
                iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
                class_ious.append(iou)
                
    # for class_id, iou in enumerate(class_ious):
    # print(f'Class {class_id} IoU: {iou:.4f}')           
   
    # mIoU 계산
    val_mIoU = np.mean(class_ious)
    
    # 에폭마다 결과 출력
    print(f"\nEpoch{epoch+1}")
    print(f"Train Loss: {epoch_loss/len(dataloader)}, Train mIoU Score: {train_mIoU:.4f}")
    print(f"Validation Loss: {val_loss/len(valid_dataloader)}, Validation mIoU Score: {val_mIoU:.4f}")
    print("___________________________________________________________________________________________\n")


100%|██████████| 30/30 [00:07<00:00,  3.75it/s]



Epoch1
Train Loss: 1.3482940067415652, Train mIoU Score: 0.1327
Validation Loss: 0.8060904145240784, Validation mIoU Score: 0.2159
___________________________________________________________________________________________



100%|██████████| 30/30 [00:08<00:00,  3.74it/s]



Epoch2
Train Loss: 0.714847924268764, Train mIoU Score: 0.2481
Validation Loss: 0.7964246869087219, Validation mIoU Score: 0.2367
___________________________________________________________________________________________



100%|██████████| 30/30 [00:08<00:00,  3.74it/s]



Epoch3
Train Loss: 0.5378639702347742, Train mIoU Score: 0.3159
Validation Loss: 0.6189639886220296, Validation mIoU Score: 0.3023
___________________________________________________________________________________________



100%|██████████| 30/30 [00:08<00:00,  3.72it/s]



Epoch4
Train Loss: 0.4498703471128491, Train mIoU Score: 0.3689
Validation Loss: 0.6849261651436488, Validation mIoU Score: 0.3029
___________________________________________________________________________________________



100%|██████████| 30/30 [00:08<00:00,  3.74it/s]


Epoch5
Train Loss: 0.38538472786329797, Train mIoU Score: 0.4188
Validation Loss: 0.6410151302814484, Validation mIoU Score: 0.3128
___________________________________________________________________________________________



In [64]:
class UNet(nn.Module):
    
    def __init__(self, num_classes):
        super(UNet, self).__init__()
        self.num_classes = num_classes
        self.contracting_11 = self.conv_block(in_channels=3, out_channels=64)
        self.contracting_12 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_21 = self.conv_block(in_channels=64, out_channels=128)
        self.contracting_22 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_31 = self.conv_block(in_channels=128, out_channels=256)
        self.contracting_32 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.contracting_41 = self.conv_block(in_channels=256, out_channels=512)
        self.contracting_42 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.middle = self.conv_block(in_channels=512, out_channels=1024)
        self.expansive_11 = nn.ConvTranspose2d(in_channels=1024, out_channels=512, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_12 = self.conv_block(in_channels=1024, out_channels=512)
        self.expansive_21 = nn.ConvTranspose2d(in_channels=512, out_channels=256, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_22 = self.conv_block(in_channels=512, out_channels=256)
        self.expansive_31 = nn.ConvTranspose2d(in_channels=256, out_channels=128, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_32 = self.conv_block(in_channels=256, out_channels=128)
        self.expansive_41 = nn.ConvTranspose2d(in_channels=128, out_channels=64, kernel_size=3, stride=2, padding=1, output_padding=1)
        self.expansive_42 = self.conv_block(in_channels=128, out_channels=64)
        self.output = nn.Conv2d(in_channels=64, out_channels=num_classes, kernel_size=3, stride=1, padding=1)
        
    def conv_block(self, in_channels, out_channels):
        block = nn.Sequential(nn.Conv2d(in_channels=in_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=out_channels),
                                    nn.Conv2d(in_channels=out_channels, out_channels=out_channels, kernel_size=3, stride=1, padding=1),
                                    nn.ReLU(),
                                    nn.BatchNorm2d(num_features=out_channels))
        return block
    
    def forward(self, X):
        contracting_11_out = self.contracting_11(X) # [-1, 64, 256, 256]
        contracting_12_out = self.contracting_12(contracting_11_out) # [-1, 64, 128, 128]
        contracting_21_out = self.contracting_21(contracting_12_out) # [-1, 128, 128, 128]
        contracting_22_out = self.contracting_22(contracting_21_out) # [-1, 128, 64, 64]
        contracting_31_out = self.contracting_31(contracting_22_out) # [-1, 256, 64, 64]
        contracting_32_out = self.contracting_32(contracting_31_out) # [-1, 256, 32, 32]
        contracting_41_out = self.contracting_41(contracting_32_out) # [-1, 512, 32, 32]
        contracting_42_out = self.contracting_42(contracting_41_out) # [-1, 512, 16, 16]
        middle_out = self.middle(contracting_42_out) # [-1, 1024, 16, 16]
        expansive_11_out = self.expansive_11(middle_out) # [-1, 512, 32, 32]
        expansive_12_out = self.expansive_12(torch.cat((expansive_11_out, contracting_41_out), dim=1)) # [-1, 1024, 32, 32] -> [-1, 512, 32, 32]
        expansive_21_out = self.expansive_21(expansive_12_out) # [-1, 256, 64, 64]
        expansive_22_out = self.expansive_22(torch.cat((expansive_21_out, contracting_31_out), dim=1)) # [-1, 512, 64, 64] -> [-1, 256, 64, 64]
        expansive_31_out = self.expansive_31(expansive_22_out) # [-1, 128, 128, 128]
        expansive_32_out = self.expansive_32(torch.cat((expansive_31_out, contracting_21_out), dim=1)) # [-1, 256, 128, 128] -> [-1, 128, 128, 128]
        expansive_41_out = self.expansive_41(expansive_32_out) # [-1, 64, 256, 256]
        expansive_42_out = self.expansive_42(torch.cat((expansive_41_out, contracting_11_out), dim=1)) # [-1, 128, 256, 256] -> [-1, 64, 256, 256]
        output_out = self.output(expansive_42_out) # [-1, num_classes, 256, 256]
        return output_out

In [67]:
# model 초기화
model = UNet(13).to(device)

# loss function과 optimizer 정의
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [68]:
# training loop
for epoch in range(5):  # 5 에폭 동안 학습합니다.
          
    # 클래스별 IoU를 누적할 리스트 초기화
    class_ious = []
    
    # 학습
    model.train()
    epoch_loss = 0
    
    for images, masks in tqdm(dataloader):
        images = images.float().to(device)
        masks = masks.long().to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks.squeeze(1))
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        # validation 클래스별 IoU 계산
        outputs = torch.softmax(outputs, dim=1).cpu()
        outputs = torch.argmax(outputs, dim=1).numpy()

        for class_id in range(num_classes):
            iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
            class_ious.append(iou)

    # mIoU 계산
    train_mIoU = np.mean(class_ious)
    #mIoU = np.mean(class_ious)
    #print(f"Epoch {epoch+1}, Train Loss: {epoch_loss/len(dataloader)}, train mIoU Score: {mIoU:.4f}")
    
    # validation
    val_loss = 0
    class_ious = []  # 클래스별 IoU를 누적할 리스트 초기화
    with torch.no_grad():
        model.eval()
        for images, masks in tqdm(valid_dataloader):
            images = images.float().to(device)
            masks = masks.long().to(device)
            outputs = model(images)

            # validation loss 계산
            val_loss += criterion(outputs, masks.squeeze(1)).item()

            # validation 클래스별 IoU 계산
            outputs = torch.softmax(outputs, dim=1).cpu()
            outputs = torch.argmax(outputs, dim=1).numpy()

            for class_id in range(num_classes):
                iou = calculate_iou_per_class(np.array(masks.cpu()), np.array(outputs), class_id)
                class_ious.append(iou)
                
    # for class_id, iou in enumerate(class_ious):
    # print(f'Class {class_id} IoU: {iou:.4f}')           
   
    # mIoU 계산
    val_mIoU = np.mean(class_ious)
    
    # 에폭마다 결과 출력
    print(f"\nEpoch{epoch+1}")
    print(f"Train Loss: {epoch_loss/len(dataloader)}, Train mIoU Score: {train_mIoU:.4f}")
    print(f"Validation Loss: {val_loss/len(valid_dataloader)}, Validation mIoU Score: {val_mIoU:.4f}")
    print("___________________________________________________________________________________________\n")

100%|██████████| 30/30 [00:08<00:00,  3.47it/s]



Epoch1
Train Loss: 0.6553465376297632, Train mIoU Score: 0.2971
Validation Loss: 0.7879598498344421, Validation mIoU Score: 0.2571
___________________________________________________________________________________________



100%|██████████| 30/30 [00:08<00:00,  3.47it/s]



Epoch2
Train Loss: 0.4347653747468755, Train mIoU Score: 0.3868
Validation Loss: 0.6730321039756139, Validation mIoU Score: 0.3099
___________________________________________________________________________________________



100%|██████████| 30/30 [00:08<00:00,  3.47it/s]



Epoch3
Train Loss: 0.36754483675611194, Train mIoU Score: 0.4323
Validation Loss: 0.5731611331303914, Validation mIoU Score: 0.3273
___________________________________________________________________________________________



100%|██████████| 30/30 [00:08<00:00,  3.46it/s]



Epoch4
Train Loss: 0.33712657789389294, Train mIoU Score: 0.4539
Validation Loss: 1.202122648557027, Validation mIoU Score: 0.2794
___________________________________________________________________________________________



100%|██████████| 30/30 [00:08<00:00,  3.46it/s]


Epoch5
Train Loss: 0.3003116465997005, Train mIoU Score: 0.4808
Validation Loss: 0.4931631540258726, Validation mIoU Score: 0.3567
___________________________________________________________________________________________



## Inference

In [54]:
test_dataset = CustomDataset(csv_file='./test.csv', transform=transform, infer=True)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=4)

In [55]:
with torch.no_grad():
    model.eval()
    result = []
    for images in tqdm(test_dataloader):
        images = images.float().to(device)
        outputs = model(images)
        outputs = torch.softmax(outputs, dim=1).cpu()
        outputs = torch.argmax(outputs, dim=1).numpy()
        # batch에 존재하는 각 이미지에 대해서 반복
        for pred in outputs:
            pred = pred.astype(np.uint8)
            pred = Image.fromarray(pred) # 이미지로 변환
            pred = pred.resize((960, 540), Image.NEAREST) # 960 x 540 사이즈로 변환
            pred = np.array(pred) # 다시 수치로 변환
            # class 0 ~ 11에 해당하는 경우에 마스크 형성 / 12(배경)는 제외하고 진행
            for class_id in range(12):
                class_mask = (pred == class_id).astype(np.uint8)
                if np.sum(class_mask) > 0: # 마스크가 존재하는 경우 encode
                    mask_rle = rle_encode(class_mask)
                    result.append(mask_rle)
                else: # 마스크가 존재하지 않는 경우 -1
                    result.append(-1)

100%|██████████| 119/119 [00:44<00:00,  2.68it/s]


## Submission

In [56]:
submit = pd.read_csv('./sample_submission.csv')
submit['mask_rle'] = result
submit

,id,mask_rle
0,TEST_0000_class_0,465717 8 466677 8 467632 18 468592 18 469527 4...
1,TEST_0000_class_1,-1
2,TEST_0000_class_2,1 159 755 365 1715 352 2675 5 2688 180 2872 15...
3,TEST_0000_class_3,483845 5 483858 4 484805 5 484818 4 485761 26 ...
4,TEST_0000_class_4,230071 9 231031 9 231995 9 232955 9 233920 8 2...
...,...,...
22771,TEST_1897_class_7,176217 4 177177 4 178137 4 181012 5 181972 5 1...
22772,TEST_1897_class_8,108 536 678 124 1068 536 1638 124 2032 528 259...
22773,TEST_1897_class_9,194461 4 195421 4 196381 4 298655 17 299615 17
22774,TEST_1897_class_10,192541 4 193501 4


In [57]:
submit.to_csv('./baseline_submit.csv', index=False)